# Video Sampling Example

This notebook demonstrates how to use the sampling functions to generate videos from text prompts.

In [1]:
import os
model_paths = [
    # "/workspace/sf-copy/merged_checkpoints_v2/merged_0.1_0.9_0.0.safetensors",
    # # "/workspace/sf-copy/merged_checkpoints_v2/merged_0.1_0.0_0.9.safetensors",
    # "/workspace/sf-copy/merged_checkpoints_v4/merged_0.05_0.0_0.95.safetensors", 
    # "/workspace/sf-copy/merged_checkpoints_v4/merged_0.05_0.95_0.0.safetensors", 
    # "/workspace/sf-copy/merged_checkpoints_v4/merged_0.05_0.8_0.15.safetensors",
    "checkpoints/14b_200.safetensors",
    # "checkpoints/14b_200.safetensors",
    # "/workspace/sf-copy/release_candidates_sf_14b/merged_0.1_0.0_0.9_bf16.sft",
]
for path in model_paths:
    
    print(os.path.exists(path))

True


In [2]:
import sys
from sample import sample_videos, sample_single_video, prompts
from release_server import load_merge_config, load_all

SageAttention loaded successfully
flash attn 2 available False
flash attn 3 available False
sage attn available True
SAGEATTN_AVAILABLE: True
DO_COMPILE False


In [3]:
import torch
import gc
config = load_merge_config("configs/self_forcing_server_14b.yaml")
# config.checkpoint_path = "/workspace/sf-copy/release_candidates_sf_14b/merged_0.1_0.0_0.9_bf16.sft"
models = load_all(config)
print("✅ Models loaded and ready to use!")
gc.collect()
torch.cuda.empty_cache()


2025-10-15 12:37:24.319 - INFO - Starting model loading...
2025-10-15 12:37:24.320 - DEBUG - Using checkpoint: checkpoints/14b_200.safetensors
Loading transformer:   0%|          | 0/4 [00:00<?, ?it/s]

2025-10-15 12:37:27.558 - DEBUG - Transformer import took: 3.23s
2025-10-15 12:37:34.280 - DEBUG - Loading transformer state dict from checkpoints/14b_200.safetensors


Loading WAN model, with name Wan2.1-T2V-14B
timestep shift,  5.0


2025-10-15 12:38:14.364 - DEBUG - Transformer load completed in: 46.81s, total: 50.04s
2025-10-15 12:38:14.366 - DEBUG - Loading transformer took: 50.04s
Loading text encoder:  25%|██▌       | 1/4 [00:50<02:30, 50.04s/it]2025-10-15 12:38:14.368 - DEBUG - Text encoder import took: 0.00s
2025-10-15 12:38:17.595 - DEBUG - Text encoder load completed in: 3.23s, total: 3.23s
2025-10-15 12:38:17.597 - DEBUG - Loading text encoder took: 3.23s
Loading VAE:  50%|█████     | 2/4 [00:53<00:45, 22.51s/it]         2025-10-15 12:38:17.953 - DEBUG - Using demo_utils.vae_block3.VAEEncoderWrapper
2025-10-15 12:38:17.953 - DEBUG - Using demo_utils.vae_block3.VAEDecoderWrapper
2025-10-15 12:38:17.999 - INFO - loading Wan-2.1/Wan2.1-T2V-1.3B/Wan2.1_VAE.pth
2025-10-15 12:38:19.453 - DEBUG - VAE load completed in: 1.50s
2025-10-15 12:38:19.492 - DEBUG - Loading VAE took: 1.54s
Initializing pipeline:  75%|███████▌  | 3/4 [00:55<00:13, 13.09s/it]2025-10-15 12:38:19.508 - DEBUG - Pipeline import took: 0.01s
20

Incompatible _IncompatibleKeys(missing_keys=['mean', 'std'], unexpected_keys=[]) while loading vae decoder
KV inference with 3 frames per block
✅ Models loaded and ready to use!


In [ ]:

import matplotlib.pyplot as plt
from release_server import get_denoising_schedule
from pprint import pprint
from wan_wrapper import FlowMatchScheduler



models.pipeline.scheduler = FlowMatchScheduler(shift=5.0, sigma_min=0.0, extra_one_step=True)
models.pipeline.scheduler.set_timesteps(1000, training=True)
st = models.pipeline.scheduler.timesteps
# print(st)
st = st.clone()
zero_padded_timesteps = torch.cat((st.clone().cpu(), torch.tensor([0], dtype=torch.float32, device="cpu"))).to("cuda")
# pprint(zero_padded_timesteps.tolist())
t = get_denoising_schedule(zero_padded_timesteps.cpu(), 1, steps=5)
# print(t)
# print(zero_padded_timesteps[-4:])
# plt.plot(zero_padded_timesteps.tolist())
# # print(type(t))
# t = torch.nn.functional.interpolate(t, size=len(zero_padded_timesteps)).squeeze(0)
# print(t.shape)
print(t)

plt.plot(t.tolist())
plt.show()


In [ ]:

test_prompts = [
    "A sorcerer stands with one hand outstretched, holding a roiling flame that coils and twists restlessly around his palm and fingers, sparks shooting off in unpredictable arcs. The fire spirals and lashes outward, wrapping around his arm like a living serpent before snapping back toward his hand in a continuous, fluid motion. His cloak ripples in the heat’s updraft as the flame flares and contracts, creating bursts of glowing embers that rise and scatter through the air",
    "Adrenaline-pumped, wide-eyed ginger kitten in vintage aviator goggles blasts down a narrow cobblestone street on a bright-yellow mini-bicycle. The camera rotates around the cat, ending on a camera shot behind its back.",
    "A stylish woman walks down a Tokyo street filled with warm glowing neon and animated city signage. She wears a black leather jacket, a long red dress, and black boots, and carries a black purse. She wears sunglasses and red lipstick. She walks confidently and casually. The street is damp and reflective, creating a mirror effect of the colorful lights. Many pedestrians walk about.",
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The static camera holds the scene firmly in place, emphasizing the intensity and grace of his movements against the stillness of the landscape.",
    "Tracking shot, cinematic style, of a surreal alien bird with iridescent feathers and bio-luminescent wings, gracefully flying through a dense alien forest — the camera follows the bird in one continuous dynamic shot, weaving through twisted glowing trees, over bioluminescent mushrooms, and past floating pollen-like orbs. The bird performs elegant aerial maneuvers — barrel rolls, sharp turns, dives — as it dodges branches and interacts with the strange atmosphere. The forest pulses with ambient light and motion-reactive flora.",
    "A single crystalline drop of water hovers in midair against a soft gradient background, shimmering with refracted light. The drop quivers, rippling with internal motion, before stretching outward as if blooming from within. Its surface tension warps into delicate translucent petals that unfurl in one continuous motion. The petals radiate with glistening reflections, the drop gracefully becoming a luminous flower suspended in space.",
    "Two scarred alley cats, one orange tabby with torn ears and one lean gray bruiser, stand upright in a flickering streetlight. They lash out with vicious swipes, their miniature boxing gloves cracked and worn, landing heavy blows that make fur and sweat fly. The gray cat snarls as it takes a hook to the jaw, stumbling back before lunging forward with a brutal counter. The sound of claws scraping the pavement mixes with the thud of their punches, the fight relentless and raw.",
    "She returns in a radiant gown made entirely of flowing gold silk, draped in voluminous layers that trail behind her in shimmering waves. The fabric ripples dramatically with each step, catching and scattering light across the runway. The dress moves like liquid metal, transforming her walk into a continuous display of luxury and opulence, radically different from her previous sculptural look.",
    "Under glowing neon lights in a surreal diner, a polar bear sits awkwardly in a red leather booth, gripping a massive BLT with mayonnaise dripping from its paws. It devours the sandwich with primal hunger, grease smearing across its white fur as the jukebox hums in the background. Fries scatter across the floor as the bear’s huge body shifts against the tiny furniture, the booth creaking under its weight. The continuous shot blends humor, absurdity, and menace in equal measure.",
    "Inside a steaming porcelain coffee cup, two tiny wooden galleons clash on dark, swirling waves of coffee. Cannons fire with sharp flashes, sending miniature plumes of smoke curling into the air, while splinters of wood spray from direct hits. The surface of the coffee sloshes violently with every broadside, waves lapping against the cup’s rim. Steam drifts upward, blending with the smoke, as the ships circle each other in a chaotic, continuous duel.",
    "A lone dancer dances under a spotlight, her movements fluid and graceful as she twirls and leaps across the stage. The camera captures every intricate detail of her movements, from the way her hair flows in the light to the way her shoes catch the stage’s polished floor. She moves with a sense of purpose and determination, her every movement carefully choreographed to create a perfect performance. The camera shows her full body",
    "A knight on horseback gallops across a windswept plain at sunset, cloak snapping in the air as dust kicks up behind the hooves. The camera tracks low beside him, the golden light glinting off his armor as the wind bends the tall grass in rhythmic waves. Underexposed, undersaturated, muted color palette, cinematic scene with professional color grading",
    "Four knights on horseback gallop across a windswept plain at sunset, flags and swords waving in the air as dust kicks up behind the hooves. Their movement is dynamic, they stand up in the saddle to wave their flags, The camera tracks low beside them, the golden light glinting off their armor as the wind bends the tall grass in rhythmic waves. Underexposed, undersaturated, muted color palette, cinematic scene with professional color grading", 
    "A cowboy spins a gun, shoots it, then blows out the smoke"
]

In [7]:
import matplotlib.pyplot as plt
from release_server import get_denoising_schedule
from pprint import pprint
from utils.wan_wrapper import FlowMatchScheduler



models.pipeline.scheduler = FlowMatchScheduler(shift=5, sigma_min=0.0, extra_one_step=True)
models.pipeline.scheduler.set_timesteps(1000, training=True)
st = models.pipeline.scheduler.timesteps
import sample
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    seed=133333,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=8,
)
prompts = [
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings",
]
results = sample.sample_videos(
    prompts_list=prompts,
    models=models,  # Reuse loaded mpropmodels
    params=params,
    output_dir="merged_8step_shift5_",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/1 [00:00<?, ?it/s]


📝 Prompt 1/1: A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings
Initializing kv cache with shape:  [1, 9360, 40, 128]
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  967.7055,  925.8299,  869.3666,  789.0856,  665.8879,
         452.8061,    0.0000], device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
frames shape:  torch.Size([1, 3, 480, 8

Sampling videos: 100%|██████████| 1/1 [00:30<00:00, 31.00s/it]

✅ Video saved to merged_8step_shift5_/prompt_000.mp4

🎉 All videos generated successfully!


In [5]:
import matplotlib.pyplot as plt
from release_server import get_denoising_schedule
from pprint import pprint
from utils.wan_wrapper import FlowMatchScheduler



models.pipeline.scheduler = FlowMatchScheduler(shift=5, sigma_min=0.0, extra_one_step=True)
models.pipeline.scheduler.set_timesteps(1000, training=True)
st = models.pipeline.scheduler.timesteps
import sample
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    seed=133333,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=5,
)
prompts = [
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The static camera holds the scene firmly in place, emphasizing the intensity and grace of his movements against the stillness of the landscape.",
]
results = sample.sample_videos(
    prompts_list=test_prompts,
    models=models,  # Reuse loaded models
    params=params,
    output_dir="merged_5step_shift5",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/13 [00:00<?, ?it/s]


📝 Prompt 1/13: A sorcerer stands with one hand outstretched, holding a roiling flame that coils and twists restlessly around his palm and fingers, sparks shooting off in unpredictable arcs. The fire spirals and lashes outward, wrapping around his arm like a living serpent before snapping back toward his hand in a continuous, fluid motion. His cloak ripples in the heat’s updraft as the flame flares and contracts, creating bursts of glowing embers that rise and scatter through the air
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization o

Sampling videos:   8%|▊         | 1/13 [00:15<03:01, 15.10s/it]

✅ Video saved to merged_5step_shift5/prompt_000.mp4

📝 Prompt 2/13: Adrenaline-pumped, wide-eyed ginger kitten in vintage aviator goggles blasts down a narrow cobblestone street on a bright-yellow mini-bicycle. The camera rotates around the cat, ending on a camera shot behind its back.
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 4
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 5
frames shape:  torch.Size

Sampling videos:  15%|█▌        | 2/13 [00:30<02:48, 15.35s/it]

...

✅ Video saved to merged_5step_shift5/prompt_010.mp4

📝 Prompt 12/13: A knight on horseback gallops across a windswept plain at sunset, cloak snapping in the air as dust kicks up behind the hooves. The camera tracks low beside him, the golden light glinting off his armor as the wind bends the tall grass in rhythmic waves. Underexposed, undersaturated, muted color palette, cinematic scene with professional color grading
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 4
frames shape

Sampling videos:  92%|█████████▏| 12/13 [03:03<00:15, 15.28s/it]

✅ Video saved to merged_5step_shift5/prompt_011.mp4

📝 Prompt 13/13: Four knights on horseback gallop across a windswept plain at sunset, flags and swords waving in the air as dust kicks up behind the hooves. Their movement is dynamic, they stand up in the saddle to wave their flags, The camera tracks low beside them, the golden light glinting off their armor as the wind bends the tall grass in rhythmic waves. Underexposed, undersaturated, muted color palette, cinematic scene with professional color grading
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  937.5000,  833.3333,  625.0000,    0.0000],
       device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 2
frames shape:  torch.Size([1, 3, 480, 832])
Zero reinitialization of kv cache
reencoding first latent frame, block idx: 3
frames shape:  torch.Size([1, 3, 480, 832])

Sampling videos: 100%|██████████| 13/13 [03:18<00:00, 15.26s/it]

✅ Video saved to merged_5step_shift5/prompt_012.mp4

🎉 All videos generated successfully!


In [9]:
from sample import create_grid
create_grid(
    outputs_folder="shiftsweep",
    grid_output_dir="shiftsweep_grids"
)

📁 Found 2 output folders: ['current_4step_ts5', 'current_release_5step_shift4']
🎬 Found 12 unique prompts

📹 Creating grid for prompt_000 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_000_grid.mp4

📹 Creating grid for prompt_001 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_001_grid.mp4

📹 Creating grid for prompt_002 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_002_grid.mp4

📹 Creating grid for prompt_003 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_003_grid.mp4

📹 Creating grid for prompt_004 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_004_grid.mp4

📹 Creating grid for prompt_005 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_005_grid.mp4

📹 Creating grid for prompt_006 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_006_grid.mp4

📹 Creating grid for prompt_007 (2 videos)
✅ Grid video saved to shiftsweep_grids/prompt_007_grid.mp4

📹 Creating grid for prompt_008 (2 videos)
✅ Grid video saved to shiftsweep_gr

{'prompt_000': PosixPath('shiftsweep_grids/prompt_000_grid.mp4'),
 'prompt_001': PosixPath('shiftsweep_grids/prompt_001_grid.mp4'),
 'prompt_002': PosixPath('shiftsweep_grids/prompt_002_grid.mp4'),
 'prompt_003': PosixPath('shiftsweep_grids/prompt_003_grid.mp4'),
 'prompt_004': PosixPath('shiftsweep_grids/prompt_004_grid.mp4'),
 'prompt_005': PosixPath('shiftsweep_grids/prompt_005_grid.mp4'),
 'prompt_006': PosixPath('shiftsweep_grids/prompt_006_grid.mp4'),
 'prompt_007': PosixPath('shiftsweep_grids/prompt_007_grid.mp4'),
 'prompt_008': PosixPath('shiftsweep_grids/prompt_008_grid.mp4'),
 'prompt_009': PosixPath('shiftsweep_grids/prompt_009_grid.mp4'),
 'prompt_010': PosixPath('shiftsweep_grids/prompt_010_grid.mp4'),
 'prompt_011': PosixPath('shiftsweep_grids/prompt_011_grid.mp4')}

## Video to Video

In [ ]:

params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=9,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=8,

    input_vid = "......" # ADD this
)
prompts = [
    "A massive wooden ship sails through a dark, stormy sea, its masts swaying violently as enormous waves crash against the hull. The ocean roars with relentless power, sending bursts of white spray over the deck as the crew struggles to keep control. Rain pours down in thick sheets, blurring the horizon into a gray haze. The camera moves with the ship’s motion, capturing the rhythm of the storm and the sheer force of the sea.",
]
results = sample.sample_videos(
    prompts_list=[p],
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_release",
    save_videos=True,
    fps=16
)